## 0. Libraries and Personal Tools

In [1]:
from dask.distributed import Client

In [2]:
client = Client() 

In [10]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42593,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: 5 minutes ago,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:42077,Total threads: 6
Dashboard: http://127.0.0.1:41869/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:34211,


In [26]:
client.close()

In [4]:
import dask.dataframe as dd

In [5]:
from pandas import read_csv
dtypes_df = read_csv("../data/raw/train_dtypes.csv")
dtypes = {k: v for (k, v) in zip(dtypes_df.column, dtypes_df.dtype)}

In [7]:
ddf = dd.read_csv('../data/raw/train_*.csv', dtype = dtypes)

In [20]:
ddf

,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_pos_z,ball_vel_x,ball_vel_y,ball_vel_z,p0_pos_x,p0_pos_y,p0_pos_z,p0_vel_x,p0_vel_y,p0_vel_z,p0_boost,p1_pos_x,p1_pos_y,p1_pos_z,p1_vel_x,p1_vel_y,p1_vel_z,p1_boost,p2_pos_x,p2_pos_y,p2_pos_z,p2_vel_x,p2_vel_y,p2_vel_z,p2_boost,p3_pos_x,p3_pos_y,p3_pos_z,p3_vel_x,p3_vel_y,p3_vel_z,p3_boost,p4_pos_x,p4_pos_y,p4_pos_z,p4_vel_x,p4_vel_y,p4_vel_z,p4_boost,p5_pos_x,p5_pos_y,p5_pos_z,p5_vel_x,p5_vel_y,p5_vel_z,p5_boost,boost0_timer,boost1_timer,boost2_timer,boost3_timer,boost4_timer,boost5_timer,player_scoring_next,team_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
npartitions=143,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float16,float32,float32,float32,float32,float32,float32,float16,float32,float32,float32,float32,float32,float32,float16,float32,float32,float32,float32,float32,float32,float16,float32,float32,float32,float32,float32,float32,float16,float32,float32,float32,float32,float32,float32,float16,float16,float16,float16,float16,float16,float16,int8,object,int8,int8
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
import sys
from os.path import abspath

from multiprocessing import cpu_count
from gc import collect

In [12]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set the default figure size and theme to display good looking matplotlib plots.
rcParams["figure.figsize"] = (10, 6)
plt.style.use("fivethirtyeight")

In [13]:
from pandas import set_option
set_option("display.max_rows", 200)
set_option("display.max_columns", 100)
set_option("display.max_colwidth", 200)

In [14]:
# add absolute path from root to sys.path to use custom modules
sys.path.insert(0, abspath('..'))

from src.models.train_model import BaseModel

## 1. Build Base Model

In [15]:
base_model = BaseModel()
base_model.read_config("../models/config.yaml")
features, target = base_model.get_data()
base_model.build_base_pipeline()

In [16]:
games_idx = base_model.data.game_num
base_model.data = ddf
# ddf = dd.from_pandas(base_model.data.reset_index(drop=True), npartitions=10)

In [17]:
base_model.base_pipeline.named_steps.keys()

dict_keys(['preprocessor'])

In [ ]:
# from pandas.core.frame import DataFrame
# DataFrame(base_model.base_pipeline.fit_transform(base_model.data)).isna().sum().sum()

## 2. Parameter Optimization

### 2.1. Split Data

In [18]:
from src.utils import create_kf_groups

from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, GroupKFold

from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from xgboost import XGBClassifier

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper, CheckpointSaver
from skopt.space import Real, Categorical, Integer

In [21]:
ddf.columns

Index(['game_num', 'event_id', 'event_time', 'ball_pos_x', 'ball_pos_y',
       'ball_pos_z', 'ball_vel_x', 'ball_vel_y', 'ball_vel_z', 'p0_pos_x',
       'p0_pos_y', 'p0_pos_z', 'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost',
       'p1_pos_x', 'p1_pos_y', 'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z',
       'p1_boost', 'p2_pos_x', 'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y',
       'p2_vel_z', 'p2_boost', 'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x',
       'p3_vel_y', 'p3_vel_z', 'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z',
       'p4_vel_x', 'p4_vel_y', 'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y',
       'p5_pos_z', 'p5_vel_x', 'p5_vel_y', 'p5_vel_z', 'p5_boost',
       'boost0_timer', 'boost1_timer', 'boost2_timer', 'boost3_timer',
       'boost4_timer', 'boost5_timer', 'player_scoring_next',
       'team_scoring_next', 'team_A_scoring_within_10sec',
       'team_B_scoring_within_10sec'],
      dtype='object')

In [23]:
from sklearn.model_selection import GroupShuffleSplit

gsp = GroupShuffleSplit(n_splits=2, test_size=0.20, random_state=777)
train_index, test_index = next(gsp.split(ddf, groups=games_idx))

ddf_train_games = ddf.loc[list(train_index), :].game_num
# train_games_idx = ddf_train_games.game_num.to_frame().compute()
# X_train = ddf[["game_num"] + features + [target]].loc[list(train_index), :]
# X_train = X_train.set_index(X_train.game_num)
# 
# X_valid = ddf[features].loc[list(test_index), :]
# y_valid = ddf[[target]].loc[list(test_index), :]


distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((subgraph_callable-dbe044d7-85f4-4f9e-a536-9880a0df2d31, [(<function read_block_from_file at 0x7f6b54c36b00>, <OpenFile '/mnt/cephfs/hadoop-compute/phoenix/ian.castillo/my-kaggle-tools/competitions/tabular-playground-series-oct-2022/notebooks/../data/raw/train_dtypes.csv'>, 0, 1132, b'\n'), None, True, True]))
kwargs:    {}
Exception: 'KeyError("None of [Index([\'game_num\'], dtype=\'object\')] are in the [columns]")'



KeyError: "None of [Index(['game_num'], dtype='object')] are in the [columns]"

In [ ]:
train_index, test_index = next(gsp.split(X_train.reset_index(drop=True).compute(), groups=train_games_idx.game_num))

X_train_cv = X_train[features].loc[train_games_idx.game_num.iloc[train_index], :]
y_train_cv = X_train[[target]].loc[train_games_idx.game_num.iloc[train_index], :]

X_test = X_train[features].loc[train_games_idx.game_num.iloc[test_index], :]
y_test = X_train[[target]].loc[train_games_idx.game_num.iloc[test_index], :]


In [ ]:
print(f"X_train_cv.compute().shape: {X_train_cv.compute().shape}")
print(f"X_test.compute().shape: {X_test.compute().shape}")
print(f"X_valid.compute().shape: {X_valid.compute().shape}")

In [ ]:
base_model.base_pipeline

In [ ]:
# from pandas.core.frame import DataFrame
# DataFrame(base_model.base_pipeline.fit_transform(X_train_cv)).describe().transpose()

In [ ]:
y_train_cv.value_counts()

In [ ]:
base_model.base_pipeline.fit(X_train_cv, y_train_cv)

In [ ]:
X_train_trans = base_model.base_pipeline.transform(X_train_cv)
X_test_trans = base_model.base_pipeline.transform(X_test)

### 2.3. Define K-Group-Folds

In [ ]:
n_folds = 5

game_num = X_train_cv.index.get_level_values("game_num")
groups = create_kf_groups(game_num, n_folds=n_folds)

gkf = GroupKFold(n_splits=n_folds)

In [ ]:
groups.value_counts()

In [ ]:
if base_model.config["model"]["ipca"]["batch_size"] == "auto":
    TOTAL_IPCA_BATCHES = 50
    ipca_batch = int(round(groups.value_counts().mean() / TOTAL_IPCA_BATCHES, -3))
else:
    ipca_batch = base_model.config["model"]["ipca"]["batch_size"]
ipca_batch

In [ ]:
base_model.base_pipeline.steps.append((
    "ipca", 
    IncrementalPCA(
        n_components=base_model.config["model"]["ipca"]["n_components"], 
        batch_size=ipca_batch,
        whiten=base_model.config["model"]["ipca"]["whiten"]
        ),
    ))


In [ ]:
if base_model.config["model"]["type"] == "xgb":
    clf = XGBClassifier(objective="binary:logistic", random_state=777)
    search_spaces = {
            "xgb__n_estimators": Integer(200, 400),
            "xgb__learning_rate": Real(0.05, 0.15, "uniform"),
            "xgb__max_depth": Integer(4, 6),
            "xgb__gamma": Real(0.05, 0.10, "uniform"),
            "xgb__subsample": Real(0.6, 0.8, "uniform"),
            "xgb__colsample_bytree": Real(0.8, 1.0, "uniform"),
        }

elif base_model.config["model"]["type"] == "lgbm":
    clf = LGBMClassifier(objective="binary", min_child_samples=None, random_state=777)


In [ ]:
base_model.base_pipeline.steps.append((base_model.config["model"]["type"], clf))

In [ ]:
base_model.base_pipeline

In [ ]:
collect()

### 2.4. Hyperparameters - Bayesian Optimization

In [ ]:
from skopt.space import Integer, Categorical, Real
from skopt.utils import use_named_args
from skopt import gp_minimize
from numpy import mean as np_mean

# -----------------------------------------------------------------------------------
#                   Guide on which params to tune/ NOT to tune
#           source: https://github.com/Microsoft/LightGBM/issues/695
# -----------------------------------------------------------------------------------
# 
# For heavily unbalanced datasets such as 1:10000:
# 
# - max_bin: keep it only for memory pressure, not to tune (otherwise overfitting)
# - learning rate: keep it only for training speed, not to tune (otherwise overfitting)
# - n_estimators: must be infinite and use early stopping to auto-tune (otherwise overfitting)
# - num_leaves: [7, 4095]
# - max_depth: [2, 63] and infinite 
# - scale_pos_weight: [1, 10000] 
# - min_child_weight: [0.01, (sample size / 1000)] 
# - subsample: [0.4, 1]
# - bagging_fraction: only 1, keep as is (otherwise overfitting)
# - colsample_bytree: [0.4, 1]
# 
# Never tune following parameters unless you have an explicit requirement to tune them:
#
# - Learning rate (lower means longer to train but more accurate, higher means smaller to train but less accurate)
# - Number of boosting iterations (automatically tuned with early stopping and learning rate)
# - Maximum number of bins (RAM dependent)

# set up hyperparameter space
space = [
    Real(0.01, 0.15, name="learning_rate"),
    Integer(7500, 10000, name="n_estimators"),
    Integer(50, 500, name="max_depth"),
    Integer(500, 5000, name="num_leaves"),
    Real(0.25, 1.0, name="subsample"),
    Real(0.25, 1.0, name="colsample_bytree"),
    Integer(100, 750, name="min_data_in_leaf"),
    Real(1, 10, name="scale_pos_weight"),
    Real(0.5, 15, name="min_child_weight"),
    ]

from sklearn.model_selection import cross_val_score
from typing import Callable

def collect_garbage() -> None:
    collect()

@use_named_args(space)
def objective(**params):
    base_model.base_pipeline["lgbm"].set_params(**params)
    return -np_mean(
        cross_val_score(
            base_model.base_pipeline["lgbm"], X_train_trans, y_train_cv, 
            cv=GroupKFold(n_splits=n_folds).split(X_train_cv, y_train_cv, groups=groups), 
            n_jobs=cpu_count(), 
            scoring="neg_log_loss", 
            fit_params={
                "eval_set": [(X_test_trans, y_test)],
                "eval_metric": "neg_log_loss",
                "callbacks": [
                    early_stopping(500),
                    log_evaluation(period=50, show_stdv=True), 
                    collect_garbage
                    ],
            }
            )
        )

In [ ]:
checkpoint_callback = CheckpointSaver("../models/optmization/checkpoints/lgbm_low_memory.pkl", compress=9)

reg_gp = gp_minimize(
    objective, space, 
    verbose=True, 
    random_state=777, n_calls=100, 
    n_random_starts=10, 
    callback=[
        checkpoint_callback,
        DeltaYStopper(
            delta=0.0005, 
            n_best=3,
            ),
        ]
    )

In [ ]:
print('best score: {}'.format(reg_gp.fun))
print('best params:')
for i, param in enumerate(space):
    print(f"{param.name}: {reg_gp.x[i]} from space: [{param.low}, {param.high}]")

In [ ]:
team = base_model.config["model"]["team"]
model = base_model.config["model"]["type"]

best_model_params = dict()
for i, param in enumerate(space):
    best_model_params[f"{param.name}"] = reg_gp.x[i]

best_model_params

### ~~2.4. Hyperparameters - Bayesian Optimization~~ (Deprecated)

In [ ]:
# Kudos to: Luca Massaron
# Source: https://www.kaggle.com/code/lucamassaron/tutorial-bayesian-optimization-with-xgboost
# https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV
# https://github.com/fmfn/BayesianOptimization
# 

# NOTE: I was never able to pass fit parameters to the model with BayesSearchCV

# import warnings
# warnings.filterwarnings("ignore")
# 
# bayes_opt = BayesSearchCV(
#     estimator=base_model.base_pipeline["lgbm"],
#     search_spaces=search_spaces, 
#     n_iter=15,
#     pre_dispatch=30,
#     n_jobs=cpu_count(), 
#     iid=False,
#     verbose=2, 
#     scoring="neg_log_loss",
#     optimizer_kwargs={'base_estimator': 'GP'},
#     fit_params={
#         "early_stopping_rounds": 10, 
#         "verbose": 1,
#         # "eval_set": [(X_trans, y_test)],
#         # "eval_names": ["valid"],
#         # "eval_metric": "neg_log_loss",
#         "callbacks": [log_evaluation(period=25, show_stdv=True)],
#         },
#     cv=GroupKFold(n_splits=n_folds).split(X_train, y_train, groups=groups),
#     random_state=777,
#     )

In [ ]:
# NOTE: Maybe this was the issue. By defining a custom function to fit the optimizer 

# import pprint
# from tabnanny import verbose
# from time import time
# 
# def report_perf(optimizer, X, y, title="model", callbacks=None):
#     """
#     A wrapper for measuring time and performances of different optmizers
#     
#     optimizer = a sklearn or a skopt optimizer
#     X = the training set 
#     y = our target
#     title = a string label for the experiment
#     """
#     start = time()
#     
#     if callbacks is not None:
#         optimizer.fit(X, y, callback=callbacks)
#     else:
#         optimizer.fit(X, y)
#         
#     d=DataFrame(optimizer.cv_results_)
#     best_score = optimizer.best_score_
#     best_score_std = d.iloc[optimizer.best_index_].std_test_score
#     best_params = optimizer.best_params_
#     
#     print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
#            + u"\u00B1"+" %.3f") % (time() - start, 
#                                    len(optimizer.cv_results_['params']),
#                                    best_score,
#                                    best_score_std))    
#     print('Best parameters:')
#     pprint.pprint(best_params)
#     print()
#     return best_params

In [ ]:
# best_params = report_perf(
#     bayes_opt, 
#     X_train_trans, y_train, 
#     "LGBM", 
#     callbacks=[
#         DeltaYStopper(delta=0.01), 
#         # DeadlineStopper(120)
#         ],
#     )

## 3. Train with All Data

In [ ]:
# from json import load
# with open(f"../models/team{team}/{model}_ipca_10perc/{model}_ipca_10perc.json", "r") as f:
#     best_model_params = load(f)

In [ ]:
base_model = BaseModel()
base_model.read_config("../models/config.yaml")
features, target = base_model.get_data()
base_model.build_base_pipeline()

if base_model.config["model"]["type"] == "xgb":
    best_model = XGBClassifier(**best_model_params, random_state=777)
elif base_model.config["model"]["type"] == "lgbm":
    best_model = LGBMClassifier(**best_model_params, min_child_samples=None, random_state=777)

base_model.base_pipeline.steps.append((
    "ipca", 
    IncrementalPCA(
        n_components=base_model.config["model"]["ipca"]["n_components"], 
        batch_size=ipca_batch,
        whiten=base_model.config["model"]["ipca"]["whiten"]
        ),
    ))

X_test_trans = base_model.base_pipeline.fit_transform(X_test)

base_model.base_pipeline.steps.append((base_model.config["model"]["type"], best_model))

In [ ]:
fit_params = {
    f"{model}__eval_set": [(X_test_trans, y_test)],
    f"{model}__eval_metric": "neg_log_loss",
    f"{model}__callbacks": [
        early_stopping(100),
        log_evaluation(period=50, show_stdv=True), 
    ],
}

fit_params.keys()

In [ ]:
if base_model.config["model"]["type"] == "xgb":
    best_model = base_model.base_pipeline.fit(X_train_cv, y_train_cv)

elif base_model.config["model"]["type"] == "lgbm":
    best_model = base_model.base_pipeline.fit(
        X=X_train_cv, y=y_train_cv, **fit_params)

## 4. Save Model

In [ ]:
best_model_params

In [ ]:
team = base_model.config["model"]["team"]
model = base_model.config["model"]["type"]

from joblib import dump
dump(best_model, f"../models/team{team}/{model}_ipca_10perc/{model}_ipca_10perc.joblib")

from json import dump, dumps
with open(f"../models/team{team}/{model}_ipca_10perc/{model}_ipca_10perc.json", "w") as f:
    dump(dumps(best_model_params, default=str), f)

## 5. Evaluate Model

In [ ]:
preds = best_model.predict_proba(X_valid)[:,1]

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_valid, preds)

## 4. Save Model

In [ ]:
import numpy as np
import seaborn as sns
from pandas.core.frame import DataFrame, Series
from pandas import concat
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve

# Kudos to: Mateus Coelho
# https://www.kaggle.com/code/mateuscco/how-to-evaluate-model-calibration/notebook

def ece(y_test, preds, strategy = 'uniform'):
    df = DataFrame({'target': y_test, 'proba': preds, 'bin': np.nan})
    
    if(strategy == 'uniform'):
        lim_inf = np.linspace(0, 0.9, 10)
        for idx, lim in enumerate(lim_inf):
            df.loc[df['proba'] >= lim, 'bin'] = idx

    elif(strategy == 'quantile'):
        pass
    
    df_bin_groups = concat([df.groupby('bin').mean(), df['bin'].value_counts()], axis = 1)
    df_bin_groups['ece'] = (df_bin_groups['target'] - df_bin_groups['proba']).abs() * (df_bin_groups['bin'] / df.shape[0])
    return df_bin_groups['ece'].sum()

def make_report(y_test, preds):
    # Computing AUC
    auc = roc_auc_score(y_test, preds)
    display(f'AUROC: {auc}')
    display(f'AUROC: {2*auc-1}')
    display(f'Fraction of positive cases in the test set: {y_test.mean()}')
    display(f'Mean predicted value in the test set:       {preds.mean()}')
    display(f'ECE (equal width bins):       {ece(y_test, preds)}')
    
    # Plotting probabilities
    display('#### Histogram of the probability distribution')
    Series(preds).hist(bins = 40)
    plt.show()
    
    # Plotting KDE by class
    display('#### KDE plots of the probability distribution by class')
    fig, ax1 = plt.subplots()
    sns.kdeplot(preds[y_test == 0], label = 'No goal', ax = ax1)
    ax2 = ax1.twinx()
    sns.kdeplot(preds[y_test == 1], label = 'Goal within 10s', color = 'red', ax = ax2)
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)
    plt.show()
    
    # Plotting calibration
    display('#### Calibration curve (equal width bins)')
    fop, mpv = calibration_curve(y_test, preds, n_bins=10)
    plt.plot(mpv, fop, "s-", label='model')
    plt.plot([0,0.25],[0,0.25], label='ideal')
    plt.xlabel('Mean predicted value')
    plt.ylabel('Fraction of positives')
    plt.legend()
    plt.show()
    
    display('#### Calibration curve (equal size bins)')
    fop, mpv = calibration_curve(y_test, preds, n_bins=10, strategy='quantile')
    plt.plot(mpv, fop, "s-", label='model')
    plt.plot([0,0.25],[0,0.25], label='ideal')
    plt.xlabel('Mean predicted value')
    plt.ylabel('Fraction of positives')
    plt.legend()
    plt.show()

In [ ]:
make_report(y_valid, preds)